# 列舉所有資產的riskiness r

In [ ]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [ ]:
rfrate = 2.45785 / 12 /100
rfrate

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [ ]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [ ]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [ ]:
def get_top5_etf(df):
    global rfrate
    
    etf_list = list(df.columns)
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        arr_returns = df[etf_name] - rfrate  
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
    #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x[:5]
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]
    
    #前5的平均
    top5_average = df[etf_top5].mean(axis=1)
    
    return etf_top5 , top5_average

# 讀取全部ETF資料夾

In [ ]:
def get_total_months(data_path):
    subdir = os.listdir(data_path)[0] #取第一個group的csv檔
    file_path = data_path + subdir + '/' + subdir + '_return.csv'
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        total_rows = df.shape[0]
        return total_rows

In [ ]:
data_path = '../2014to2018RE_data/'
# 以程式取得資料總筆數
total_data = get_total_months(data_path)
# 或是寫死都可
#total_data = 61
# 指定用來計算指標的資料個數
window_size = 30
rolling_times = total_data - window_size

In [ ]:
for i in range(rolling_times):
    print('=== Rolling #',i+1,'===')
    allgroup_top5_return = DataFrame()
    allgroup_top5_dict = dict()
    for subdir in os.listdir(data_path):
        subdir_path = os.path.join(data_path, subdir)
        if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
            file = subdir + '_return.csv'
            file_path = os.path.join(subdir_path, file)
            if os.path.isfile(file_path):
                df = pd.read_csv(file_path)
                df = df.iloc[i:i+30,2:]
                # 提出[前10名的名稱,前10名的平均各期報酬]
                [ top5_etf,allgroup_top5_return[subdir] ] = get_top5_etf(df)
                print('Group:',subdir)
                print('Top5:',top5_etf)
                print('================')
                allgroup_top5_dict[subdir] = top5_etf
                df = pd.read_csv(file_path)
                etf_date = df['Date']

In [ ]:
allgroup_top5_return = pd.concat([etf_date,allgroup_top5_return],axis=1)

In [ ]:
allgroup_top5_return.head()

# 將allgroup_top5 先存取成CSV

In [ ]:
allgroup_top5_return.to_csv("allgroup_top5_return_new.csv")

# 再次計算Riskiness R，並依照R值排序，傳回前五名的Group

In [ ]:
def get_top5_Group(data_file):

    df1 = pd.read_csv(data_file)
    etf_list1 = list(df1.columns)[2:] #<----------------------------------------------加上日期
    df1 = df1.iloc[:,2:]

    etf_dic1 = {}
    for etf_name in etf_list1:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        
        
        #-------------------------------------------------------------------------------
        
        #同上要加上時間
        
        
        arr_returns1 = df1[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        
        
        
        #---------------------------------------------------------------------------------
        
        guess1 = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic1[etf_name] = get_riskiness_r(guess1,arr_returns1)
    
    
    sorted_x1 = sorted(etf_dic1.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x1[:5]
        # top5 = sorted_reversed_x[:5]
    #print(top5)
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]

    df2 = df1[etf_top5]
    
    return df2

In [ ]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [ ]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top5_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top5_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [ ]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

# 依次計算32組的權重

In [ ]:
a = list(range(0, 32))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

# 計算出group中 前五名的權重

In [ ]:
df2 = get_top5_Group("C:\\Users\\USER\\github\\HW1--data-mining\\FinalProject\\Scripts\\allgroup_top5_return_new.csv")

In [ ]:
df2.head()

In [ ]:
a = list(range(0, 5))
all_return = df2.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

In [ ]:
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)